# Exercício de fixação de conceitos 1

## Objetivo

O objetivo desse Jupyter Notebook é exercitar conceitos de Aprendizado de Máquina ligados essencialmente à Regressão Linear, dentre eles:

- K-fold
- Cross Validation
- Critério de Root Mean Square Error (RMSE)
- Regressor Linear ótimo em forma fechada

## Problema abordado

O problema a ser trabalhado consite em predizer o número de manchas solares em uma série temporal. O datasheet fornecido contém dados diários de manchas solares desde o ano de 1749 até 2019.

## Visualização dos dados


In [8]:
import pandas as pd
import math
import numpy as np
from numpy import genfromtxt
from numpy import linalg
import matplotlib.pyplot as plt 
from scipy.linalg import toeplitz

ModuleNotFoundError: No module named 'pandas'

In [ ]:
manchas = genfromtxt('monthly-sunspots.csv', delimiter=',')

In [ ]:
manchas.shape

Retirando as colunas de índice e data.

In [ ]:
manchas = np.delete(manchas,np.s_[0:1],axis=1)


In [ ]:
manchas = np.delete(manchas,0,axis=0)

In [ ]:
plt.plot(manchas)
plt.title("Manchas solares ao longo dos meses")
plt.show()

## Parte 1 



In [ ]:
K = 3
X = toeplitz(manchas[K-1:],manchas[K-1:]) # matriz de entrada
y = manchas[K:] # matriz de saída
N = len(y) # number of training examples

print(X)

In [3]:
import sys
sys.executable

'C:\\Users\\Arlindo\\AppData\\Local\\Programs\\Python\\Python39\\python.exe'